# İntro

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import glob
path_dataset_1=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-1/train_png/*.png")
path_dataset_2=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-2/train_png/*.png")
path_dataset_3=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-3/train_png/*.png")
path_dataset_4_1=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-4-1/train_png/*.png")

path_dataset_4_2=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-4-2/train_png/*.png")
path_dataset_5_1=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-5-1-index-yok/train_png/*.png")

path_dataset_5_2=glob.glob("C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-5-2/train_png/*.png")

In [8]:
train_png_paths=path_dataset_1+path_dataset_2+path_dataset_3+path_dataset_4_1+path_dataset_4_2+path_dataset_5_1+path_dataset_5_2
len(train_png_paths)

518

In [9]:
train_png_paths[0]

'C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/dataset-1/train_png\\026c97_0.png'

In [10]:
train_png_paths[0].split('\\')[-1][:-4]

'026c97_0'

In [11]:
import numpy as np
DATASET_FOLDER = "C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/"


path_csv = os.path.join(DATASET_FOLDER, "train.csv")
df_train = pd.read_csv(path_csv)
list_df=[]
for i in df_train.index:
    for ii in range(len(train_png_paths)):
        if df_train['image_id'][i]==train_png_paths[ii].split('\\')[-1][:-4]:
            list_df.append([train_png_paths[ii],df_train['label'][i]])

In [12]:
df=pd.DataFrame(list_df,columns=['image_path','label'])
df

,image_path,label
0,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
1,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
2,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
3,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
4,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,LAA
...,...,...
513,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,LAA
514,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
515,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE
516,C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...,CE


In [13]:
df_copy=df.copy()
Fseries = pd.Series(df_copy.image_path, name="filepaths")

Lseries = pd.Series(df_copy.label, name="labels")
inme_data = pd.concat([Fseries,Lseries], axis=1)
inmer_df = pd.DataFrame(inme_data)
print(inmer_df.head())
print(inmer_df["labels"].value_counts())

#shape of datatset
inmer_df.shape

                                           filepaths labels
0  C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...     CE
1  C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...     CE
2  C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...     CE
3  C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...     CE
4  C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Da...    LAA
CE     392
LAA    126
Name: labels, dtype: int64


(518, 2)

In [14]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
#tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dense, Dropout,Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

from tensorflow.keras.callbacks import TensorBoard,EarlyStopping 

In [16]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(inmer_df, test_size=0.2, shuffle=True)


In [17]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)
train = image_gen.flow_from_dataframe(dataframe= train_df,x_col="filepaths",y_col="labels",
                                      target_size=(128,128),

                                      class_mode='categorical', #used for Sequential Model
                                      batch_size=1,
                                      shuffle=False            #do not shuffle data
                                     )
test = image_gen.flow_from_dataframe(dataframe= test_df,x_col="filepaths",y_col="labels",
                                      target_size=(128,128),

                                      class_mode='categorical', #used for Sequential Model
                                      batch_size=1,
                                      shuffle=False            #do not shuffle data
                                     )

Found 414 validated image filenames belonging to 2 classes.
Found 104 validated image filenames belonging to 2 classes.


In [18]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
IMG_SIZE=(256,256)
vgg16_weight_path = 'C:/Users/omerf/OneDrive/Desktop/Mayo_clinic/Dataset/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = VGG16(
    weights=vgg16_weight_path,
    include_top=False, 
    input_shape=IMG_SIZE + (3,)
)
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
NUM_CLASSES = 2
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
#model.add(layers.Flatten())

#model.add(layers.Dropout(0.5))

model.add(layers.Dense(NUM_CLASSES, activation='sigmoid'))



model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=1e-4),
    metrics=['accuracy']
)

model.summary()

In [ ]:
import time
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
start = time.time()
History = model.fit(train, epochs=10,verbose=1)
print("Total time: ", time.time() - start, "seconds")

In [15]:
!pip install -U efficientnet

  Attempting uninstall: efficientnet
    Found existing installation: efficientnet 1.0.0
    Uninstalling efficientnet-1.0.0:
      Successfully uninstalled efficientnet-1.0.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
segmentation-models 1.0.1 requires efficientnet==1.0.0, but you have efficientnet 1.1.1 which is incompatible.


In [13]:
import efficientnet.keras as efn

In [14]:
base_model = efn.EfficientNetB7(input_shape = (128, 128, 3), include_top = False, weights = 'imagenet')
for layer in base_model.layers:
    layer.trainable = False


In [15]:
from tensorflow.keras import Model 
x = base_model.output
x = Flatten()(x)
x = Dense(32, activation="relu")(x)
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = Dense(2, activation="sigmoid")(x)
model_final = Model(base_model.input, predictions)


In [16]:
import tensorflow
model_final.compile(tensorflow.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])

In [20]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard,LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min', 
                              verbose=1, 
                              patience=30
                             )
# save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath="class_model_EfficientNetB7.h5", 
                               verbose=1, 
                               save_best_only=True
                              )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=10,
                              min_delta=0.0001,
                              factor=0.2
                             )

In [21]:
import time
tf.config.run_functions_eagerly(True)
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
start = time.time()
eff_history = model_final.fit_generator(train, epochs = 25, steps_per_epoch=100,validation_data=test,
                                        callbacks = [checkpointer, earlystopping, reduce_lr])
print("Total time: ", time.time() - start, "seconds")

Epoch 1/25
100/100 [==============================] - ETA: 0s - loss: 0.7688 - accuracy: 0.6600
Epoch 1: val_loss improved from inf to 0.51288, saving model to class_model_EfficientNetB7.h5
100/100 [==============================] - 1088s 11s/step - loss: 0.7688 - accuracy: 0.6600 - val_loss: 0.5129 - val_accuracy: 0.8077 - lr: 1.0000e-04
Epoch 2/25
100/100 [==============================] - ETA: 0s - loss: 0.9089 - accuracy: 0.6600
Epoch 2: val_loss did not improve from 0.51288
100/100 [==============================] - 1134s 11s/step - loss: 0.9089 - accuracy: 0.6600 - val_loss: 0.6918 - val_accuracy: 0.8077 - lr: 1.0000e-04
Epoch 3/25
100/100 [==============================] - ETA: 0s - loss: 0.7246 - accuracy: 0.7200
Epoch 3: val_loss did not improve from 0.51288
100/100 [==============================] - 1064s 11s/step - loss: 0.7246 - accuracy: 0.7200 - val_loss: 0.6889 - val_accuracy: 0.8077 - lr: 1.0000e-04
Epoch 4/25
100/100 [==============================] - ETA: 0s - loss: 0